<a href="https://colab.research.google.com/github/Milewczik/naiprojekt/blob/main/projeklt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision

In [ ]:
!pip install kaggle

In [ ]:
!kaggle kernels pull gauravduttakiit/mushrooms-images-classification

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.11/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


In [ ]:
from google.colab import files
files.upload()  # Prześlij plik kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"virusxar","key":"4f579cc66c5a8df178dbdef17af581f8"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d daniilonishchenko/mushrooms-images-classification-215

Dataset URL: https://www.kaggle.com/datasets/daniilonishchenko/mushrooms-images-classification-215
License(s): ODC Public Domain Dedication and Licence (PDDL)
 99% 1.53G/1.55G [00:09<00:00, 248MB/s]
100% 1.55G/1.55G [00:09<00:00, 171MB/s]


In [ ]:
!unzip mushrooms-images-classification-215.zip -d ./mushrooms

Archive:  mushrooms-images-classification-215.zip
  inflating: ./mushrooms/data/data/almond_mushroom/0.png  
  inflating: ./mushrooms/data/data/almond_mushroom/1.png  
  inflating: ./mushrooms/data/data/almond_mushroom/10.png  
  inflating: ./mushrooms/data/data/almond_mushroom/11.png  
  inflating: ./mushrooms/data/data/almond_mushroom/2.png  
  inflating: ./mushrooms/data/data/almond_mushroom/3.png  
  inflating: ./mushrooms/data/data/almond_mushroom/4.png  
  inflating: ./mushrooms/data/data/almond_mushroom/5.png  
  inflating: ./mushrooms/data/data/almond_mushroom/6.png  
  inflating: ./mushrooms/data/data/almond_mushroom/7.png  
  inflating: ./mushrooms/data/data/almond_mushroom/8.png  
  inflating: ./mushrooms/data/data/almond_mushroom/9.png  
  inflating: ./mushrooms/data/data/amanita_gemmata/0.png  
  inflating: ./mushrooms/data/data/amanita_gemmata/1.png  
  inflating: ./mushrooms/data/data/amanita_gemmata/10.png  
  inflating: ./mushrooms/data/data/amanita_gemmata/11.png  
  

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision import models, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Używane urządzenie: {device}")

# Ścieżki do danych
train_dir = "./mushrooms/data/data"
val_dir = "./mushrooms/data/data"

batch_size = 32
epochs = 60
learning_rate = 0.001
num_classes = 215

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_data = datasets.ImageFolder(train_dir, transform=transform)
val_data = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

print("Ładowanie modelu ResNet50...")
model = models.resnet50(pretrained=True)

model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Trening - strata: {epoch_loss:.4f}")

        model.eval()
        val_loss = 0.0
        correct = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

                _, preds = torch.max(outputs, 1)
                correct += torch.sum(preds == labels)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = correct.double() / len(val_loader.dataset)
        print(f"Walidacja - strata: {val_loss:.4f}, dokładność: {val_acc:.4f}")

    print("Trening zakończony.")

if __name__ == "__main__":
    train_model(model, train_loader, val_loader, criterion, optimizer, epochs)

    torch.save(model.state_dict(), "mushroom_classifier.pth")
    print("Model zapisany jako mushroom_classifier.pth")


Używane urządzenie: cpu
Ładowanie modelu ResNet50...


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 59.9MB/s]


Epoch 1/60


KeyboardInterrupt: 

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

model_path = "mushroom_classifier.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 215)
state_dict = torch.load(model_path, map_location=device, weights_only=True)
model.load_state_dict(state_dict)
model = model.to(device)
model.eval()

classes = [
    "almond_mushroom", "amanita_gemmata", "amethyst_chanterelle", "amethyst_deceiver", "aniseed_funnel",
    "ascot_hat", "bay_bolete", "bearded_milkcap", "beechwood_sickener", "beefsteak_fungus",
    "birch_polypore", "birch_woodwart", "bitter_beech_bolete", "bitter_bolete", "black_bulgar",
    "black_morel", "blackening_brittlegill", "blackening_polypore", "blackening_waxcap", "blue_roundhead",
    "blushing_bracket", "blushing_rosette", "blushing_wood_mushroom", "bovine_bolete", "bronze_bolete",
    "brown_birch_bolete", "brown_rollrim", "bruising_webcap", "butter_cap", "cauliflower_fungus",
    "cedarwood_waxcap", "chanterelle", "charcoal_burner", "chestnut_bolete", "chicken_of_the_woods",
    "cinnamon_bracket", "clouded_agaric", "clustered_domecap", "common_bonnet", "common_inkcap",
    "common_morel", "common_puffball", "common_rustgill", "crimped_gill", "crimson_waxcap",
    "cucumber_cap", "curry_milkcap", "deadly_fibrecap", "deadly_webcap", "deathcap",
    "deer_shield", "destroying_angel", "devils_bolete", "dog_stinkhorn", "dryads_saddle",
    "dusky_puffball", "dyers_mazegill", "earthballs", "egghead_mottlegill", "elfin_saddle",
    "fairy_ring_champignons", "false_chanterelle", "false_deathcap", "false_morel", "false_saffron_milkcap",
    "fenugreek_milkcap", "field_blewit", "field_mushroom", "fleecy_milkcap", "fly_agaric",
    "fools_funnel", "fragrant_funnel", "freckled_dapperling", "frosted_chanterelle", "funeral_bell",
    "geranium_brittlegill", "giant_funnel", "giant_puffball", "glistening_inkcap", "golden_bootleg",
    "golden_scalycap", "golden_waxcap", "greencracked_brittlegill", "grey_knight", "grey_spotted_amanita",
    "grisettes", "hairy_curtain_crust", "heath_waxcap", "hedgehog_fungus", "hen_of_the_woods",
    "honey_fungus", "hoof_fungus", "horn_of_plenty", "horse_mushroom", "inky_mushroom",
    "jelly_ears", "jelly_tooth", "jubilee_waxcap", "king_alfreds_cakes", "larch_bolete",
    "leccinum_albostipitatum", "liberty_cap", "lilac_bonnet", "lilac_fibrecap", "lions_mane",
    "lurid_bolete", "macro_mushroom", "magpie_inkcap", "meadow_waxcap", "medusa_mushroom",
    "morel", "mosaic_puffball", "oak_bolete", "oak_mazegill", "oak_polypore",
    "ochre_brittlegill", "old_man_of_the_woods", "orange_birch_bolete", "orange_bolete", "orange_grisette",
    "orange_peel_fungus", "oyster_mushroom", "pale_oyster", "panthercap", "parasol",
    "parrot_waxcap", "pavement_mushroom", "penny_bun", "peppery_bolete", "pestle_puffball",
    "pine_bolete", "pink_waxcap", "plums_and_custard", "poison_pie", "poplar_bell",
    "poplar_fieldcap", "porcelain_fungus", "powdery_brittlegill", "purple_brittlegill", "red_belted_bracket",
    "red_cracking_bolete", "root_rot", "rooting_bolete", "rooting_shank", "rosy_bonnet",
    "ruby_bolete", "saffron_milkcap", "scaly_wood_mushroom", "scarlet_caterpillarclub", "scarlet_elfcup",
    "scarlet_waxcap", "scarletina_bolete", "semifree_morel", "sepia_bolete", "shaggy_bracket",
    "shaggy_inkcap", "shaggy_parasol", "shaggy_scalycap", "sheathed_woodtuft", "silky_rosegill",
    "silverleaf_fungus", "slender_parasol", "slimy_waxcap", "slippery_jack", "smoky_bracket",
    "snakeskin_grisette", "snowy_waxcap", "spectacular_rustgill", "splendid_waxcap", "splitgill",
    "spotted_toughshank", "spring_fieldcap", "st_georges_mushroom", "stinkhorn", "stinking_dapperling",
    "stubble_rosegill", "stump_puffball", "suede_bolete", "sulphur_tuft", "summer_bolete",
    "tawny_funnel", "tawny_grisette", "terracotta_hedgehog", "the_blusher", "the_deceiver",
    "the_goblet", "the_miller", "the_prince", "the_sickener", "thimble_morel",
    "tripe_fungus", "trooping_funnel", "truffles", "tuberous_polypore", "turkey_tail",
    "velvet_shank", "vermillion_waxcap", "warted_amanita", "weeping_widow", "white_dapperling",
    "white_domecap", "white_false_death_cap", "white_fibrecap", "white_saddle", "winter_chanterelle",
    "wood_blewit", "wood_mushroom", "woodland_inkcap", "woolly_milkcap", "wrinkled_peach",
    "yellow_false_truffle", "yellow_foot_waxcap", "yellow_stagshorn", "yellow_stainer", "yellow_swamp_brittlegill"
]


# Transformacje obrazu (takie same jak przy trenowaniu modelu)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def predict_top_3(image_path, model, classes):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        probabilities = torch.softmax(outputs, dim=1)

    top_probs, top_idxs = torch.topk(probabilities, 3)
    top_probs = top_probs.cpu().numpy().flatten()
    top_idxs = top_idxs.cpu().numpy().flatten()

    predictions = [(classes[idx], prob) for idx, prob in zip(top_idxs, top_probs)]
    return predictions


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [ ]:
# Ścieżka do zdjęcia
#image_path = "/content/mushrooms/data/data/bitter_beech_bolete/0.png"

image_path = "sromotnikowy.png"

predictions = predict_top_3(image_path, model, classes)

for class_name, probability in predictions:
    print(f"Class: {class_name}, Probability: {probability:.4f}")

Class: false_deathcap, Probability: 0.8085
Class: fleecy_milkcap, Probability: 0.0878
Class: warted_amanita, Probability: 0.0403
